In [1]:
import numpy as np
import numpy.random as random
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install DAWG

  Using cached https://files.pythonhosted.org/packages/a3/33/fff9675c68b5f6c63ec8c6e6ff57827dda28a1fa5b2c2d727dffff92dd47/pymorphy2-0.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/6a/84/ff1ce2071d4c650ec85745766c0047ccc3b5036f1d03559fd46bb38b5eeb/DAWG_Python-0.7.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/02/51/2465fd4f72328ab50877b54777764d928da8cb15b74e2680fc1bd8cb3173/pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python2.7/dist-packages/DAWG_Python-0.7.2.dist-info'
Consider using the `--user` option or check the permissions.

  Using cached https://files.pythonhosted.org/packages/02/51/2465fd4f72328ab50877b54777764d928da8cb15b74e2680fc1bd8cb3173/pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python2.7

In [3]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [4]:
#from google.colab import drive
#drive.mount('/gdrive')
#GPREFIX = '/gdrive/My Drive/antivulg/'
GPREFIX = ''

In [5]:
import string

punct_marks = tuple('.,!?;"\'')
english = tuple(['ENG']) + tuple(string.ascii_lowercase)
digits = tuple(['DIG']) + tuple(string.digits)
russian = tuple('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
real_alphabet = russian + tuple(' -.*')
alphabet = real_alphabet + tuple(['NULL', 'DIG', 'ENG'])

alpha2idx = {c: idx for idx, c in enumerate(alphabet)}
null_idx = alpha2idx['NULL']

reduction = {c: c for c in alphabet}
for lst in punct_marks, english, digits:
    for c in lst:
        reduction[c] = lst[0]
        if c.lower():
            reduction[c.upper()] = lst[0]
for c in russian:
    reduction[c.upper()] = c

def enc(c):
    return alpha2idx[reduction.get(c, '*')]

In [6]:
import re

book = []
for name in 'vim.txt', 'gul.txt':
    with open(GPREFIX + name) as f:
        book.extend(re.split(r'\W+', f.read()))

supervised = []
with open(GPREFIX + 'supervised.txt') as f:
    for idx, line in enumerate(f):
        if idx % 2 == 0:
            text = line[:-1]
        else:
            vulg = np.array(list(map(int, line[:-1])))
            supervised.append((text, vulg))
supervised_text = []
supervised_vulg = []
for text, vulg in supervised:
    for occ in re.finditer(r'\w+', text):
        left, right = occ.span()
        supervised_text.append(text[left:right])
        supervised_vulg.append(vulg[left:right])

norm_words = []
vulg_words = []
words = pd.read_csv(GPREFIX + 'train_words.csv')
for row in words.itertuples():
    if row.label == 0:
        norm_words.append(row.word)
    else:
        vulg_words.append(row.word)

In [7]:
def mutate_word(word):
    mutations = morph.parse(word)[0].lexeme
    return mutations[random.randint(len(mutations))].word

def do_char_repl(word, prob):
    new_word = []
    for c in word:
        if random.uniform() < prob:
            new_word.append(random.choice(real_alphabet))
        else:
            new_word.append(c)
    return ''.join(new_word)

def do_char_swap(word):
    new_word = []
    idx = 0
    while idx < len(word):
        if idx + 1 < len(word) and random.uniform() < 1 / len(word):
            new_word.append(word[idx + 1])
            new_word.append(word[idx])
            idx += 2
        else:
            new_word.append(word[idx])
            idx += 1
    return ''.join(new_word)

def get_punct(full_stop_prob=0.9, max_len=3):
    if random.uniform() < full_stop_prob:
        return '.' * random.randint(1, max_len + 1)
    return '-'

def generate_trash():
    word = []
    for _ in range(random.randint(1, 6)):
        mode = random.choice(['ENG', 'DIG', 'RUS', '.'])
        if mode == 'ENG':
            c = random.choice(english[1:])
        elif mode == 'DIG':
            c = random.choice(digits[1:])
        elif mode == 'RUS':
            c = random.choice(russian)
        else:
            c = random.choice(punct_marks)
        word.extend([c] * random.randint(1, 4))
    return ''.join(word)

def generate_text(min_words=20, max_words=40, do_mutate_word=True, book_prob=0.5, norm_prob=0.1,
                  vulg_prob=0.1, trash_prob=0.1, space_prob=0.75, punct_prob=0.2, char_repl_prob=0.05):
    num_words = random.randint(min_words, max_words + 1)
    if random.uniform() < book_prob:
        source = book
    else:
        source = supervised_text
    source_ptr = random.randint(len(source) - num_words + 1)
    text = []
    is_vulg = []
    
    def append(token, cur_vulg):
        if isinstance(cur_vulg, bool):
            is_vulg.extend([1 if cur_vulg else 0] * len(token))
        else:
            if len(token) != len(cur_vulg):
                if sum(cur_vulg) == 0:
                    cur_vulg = np.zeros(len(token))
                elif sum(cur_vulg) == len(cur_vulg):
                    cur_vulg = np.ones(len(token))
                else:
                    return
            is_vulg.extend(cur_vulg)
        text.append(token)
    
    while len(text) < num_words:
        word_outcome = random.uniform()
        if word_outcome < norm_prob:
            word = random.choice(norm_words)
            cur_vulg = False
        elif word_outcome < norm_prob + vulg_prob:
            word = random.choice(vulg_words)
            cur_vulg = True
        elif word_outcome < norm_prob + vulg_prob + trash_prob:
            word = generate_trash()
            cur_vulg = False
        else:
            word = source[source_ptr]
            if source is book:
                cur_vulg = False
            else:
                cur_vulg = supervised_vulg[source_ptr]
            source_ptr += 1

        if do_mutate_word:
            word = mutate_word(word)
        word = do_char_repl(word, char_repl_prob)
        #word = do_char_swap(word)
        append(word, cur_vulg)
        
        sep_outcome = random.uniform()
        if sep_outcome < space_prob:
            append(' ', False)
        elif sep_outcome < space_prob + punct_prob:
            append(get_punct(), False)
            
    return ''.join(text), np.array(is_vulg)

In [8]:
Y_NULL = 2

def encode_text(text):
    encoding = []
    for c in text:
        encoding.append(enc(c))
    return np.array(encoding)

def encode_batch(batch, with_y=True):

    def first_or_whole(x):
        if isinstance(x, (list, tuple)):
            return x[0]
        return x
    
    max_len = max(len(first_or_whole(sample)) for sample in batch)
    X = np.full([len(batch), max_len], null_idx)
    if with_y:
        y = np.full_like(X, Y_NULL)
    for idx, sample in enumerate(batch):
        sample_len = len(first_or_whole(sample))
        X[idx, :sample_len] = encode_text(first_or_whole(sample))
        if with_y:
            y[idx, :sample_len] = sample[1]
    if with_y:
        return X, y
    return X

In [9]:
def custom_loss_function(y_true, y_pred):
    EPS = 1e-10
    weight = tf.cast(tf.not_equal(y_true, Y_NULL), dtype=tf.float32)
    loss = -weight * (y_true * tf.log(y_pred + EPS) + (1 - y_true) * tf.log(1 - y_pred + EPS))
    return tf.reduce_mean(loss)

In [10]:
def build_model(lr=1e-2, emb_size=8, hid_size=256, kernel_size=15):
    input_layer = keras.layers.Input(shape=[None])
    
    pad_size = (kernel_size - 1) // 2
    pad_input_layer = keras.layers.Lambda(lambda x: tf.pad(x, paddings=[[0, 0], [pad_size, pad_size]],
                                                           constant_values=null_idx))(input_layer)
    
    pad_embedding_layer = keras.layers.Embedding(input_dim=len(alphabet),
                                                 output_dim=emb_size)(pad_input_layer)
    
    #pad_lstm_layer = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(units=hid_size // 2,
    #                                            return_sequences=True))(pad_embedding_layer)
    pad_lstm_layer = keras.layers.Bidirectional(keras.layers.LSTM(units=hid_size // 2,
                                                return_sequences=True))(pad_embedding_layer)
    lstm_layer = keras.layers.Lambda(lambda x: x[:, pad_size:-pad_size])(pad_lstm_layer)
    
    pad_conv_layer = keras.layers.Conv1D(filters=2 * emb_size, kernel_size=kernel_size,
                                         padding='same')(pad_embedding_layer)
    conv_layer = keras.layers.Lambda(lambda x: x[:, pad_size:-pad_size])(pad_conv_layer)
    
    concat_layer = keras.layers.Concatenate(axis=-1)([lstm_layer, conv_layer])
    batch_layer = keras.layers.BatchNormalization()(concat_layer)
    
    dense_layer = keras.layers.TimeDistributed(
        keras.layers.Dense(units=hid_size, activation='elu'))(batch_layer)
    
    output_layer = keras.layers.TimeDistributed(
        keras.layers.Dense(units=1, activation='sigmoid'))(dense_layer)
    squeezed_output_layer = keras.layers.Lambda(lambda x: tf.squeeze(
        x, axis=-1))(output_layer)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[squeezed_output_layer])
    model.compile(optimizer=keras.optimizers.Adam(lr=lr), loss=custom_loss_function, metrics=[])
    return model

def generator(batch_size, **kwargs):
    while True:
        batch = [generate_text(**kwargs) for _ in range(batch_size)]
        yield encode_batch(batch)

In [75]:
model = build_model()
model.fit_generator(generator(batch_size=32), steps_per_epoch=100, epochs=50)

Epoch 1/50
100/100 [==============================] - 66s 656ms/step - loss: 0.1879
Epoch 2/50
100/100 [==============================] - 59s 590ms/step - loss: 0.1477
Epoch 3/50
100/100 [==============================] - 58s 581ms/step - loss: 0.1172
Epoch 4/50
100/100 [==============================] - 60s 603ms/step - loss: 0.0891
Epoch 5/50
100/100 [==============================] - 59s 592ms/step - loss: 0.0751
Epoch 6/50
100/100 [==============================] - 58s 584ms/step - loss: 0.0702
Epoch 7/50
100/100 [==============================] - 57s 574ms/step - loss: 0.0643
Epoch 8/50
100/100 [==============================] - 58s 583ms/step - loss: 0.0572
Epoch 9/50
100/100 [==============================] - 58s 577ms/step - loss: 0.0563
Epoch 10/50
100/100 [==============================] - 58s 578ms/step - loss: 0.0520
Epoch 11/50
100/100 [==============================] - 59s 585ms/step - loss: 0.0461
Epoch 12/50
100/100 [==============================] - 59s 595ms/step - lo

In [76]:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss=custom_loss_function, metrics=[])
model.fit_generator(generator(batch_size=32), steps_per_epoch=100, epochs=20)

Epoch 1/20
100/100 [==============================] - 58s 579ms/step - loss: 0.0172
Epoch 2/20
100/100 [==============================] - 57s 565ms/step - loss: 0.0173
Epoch 3/20
100/100 [==============================] - 58s 576ms/step - loss: 0.0170
Epoch 4/20
100/100 [==============================] - 60s 598ms/step - loss: 0.0156
Epoch 5/20
100/100 [==============================] - 58s 582ms/step - loss: 0.0157
Epoch 6/20
100/100 [==============================] - 66s 659ms/step - loss: 0.0157
Epoch 7/20
100/100 [==============================] - 60s 601ms/step - loss: 0.0153
Epoch 8/20
100/100 [==============================] - 74s 741ms/step - loss: 0.0147
Epoch 9/20
100/100 [==============================] - 70s 700ms/step - loss: 0.0142
Epoch 10/20
100/100 [==============================] - 65s 654ms/step - loss: 0.0157
Epoch 11/20
100/100 [==============================] - 69s 689ms/step - loss: 0.0162
Epoch 12/20
100/100 [==============================] - 75s 748ms/step - lo

In [77]:
from keras.models import load_model

model.save(GPREFIX + 'model_final.h5')
#model = load_model(GPREFIX + 'model.h5',
#                   custom_objects={'tf': tf, 'custom_loss_function': custom_loss_function})

In [78]:
val = []
with open(GPREFIX + 'validation.txt') as f:
    for idx, line in enumerate(f):
        if idx % 2 == 0:
            text = line[:-1]
        else:
            vulg = np.array(list(map(int, line[:-1])))
            val.append((text, vulg))

In [79]:
val_enc, val_true = encode_batch(val)
val_pred = model.predict(val_enc)

In [97]:
from sklearn.metrics import f1_score

THRESHOLD = 0.75
val_cls = (val_pred >= THRESHOLD).astype(np.int)

print('Char-level F1:', f1_score(val_true[val_true != Y_NULL], val_cls[val_true != Y_NULL]))

tok_true = []
tok_cls = []
for (sent, true), pred in zip(val, val_pred):
    for occ in re.finditer(r'\w+', sent):
        left, right = occ.span()
        tok_true.append(1 if true[left:right].mean() >= 0.5 else 0)
        tok_cls.append(1 if pred[left:right].mean() >= THRESHOLD else 0)

print('Token-level F1:', f1_score(tok_true, tok_cls))

Char-level F1: 0.8194703468854905
Token-level F1: 0.8421052631578948


In [98]:
from IPython.display import HTML, display_html

test = [e[0] for e in val[1::10]]
test_enc = encode_batch(test, with_y=False)
test_pred = model.predict(test_enc)

def get_raw_html(text, pred):
    cmap = plt.get_cmap("Reds")

    def get_color_hex(p):
        rgba = cmap(p, bytes=True)
        return '#%02X%02X%02X' % rgba[:3]

    raw_html = []
    template = '<span style="background-color: {color_hex}">{token}</span>'
    for c, p in zip(text, pred):
        raw_html.append(template.format(color_hex=get_color_hex(p), token=c))
    return ''.join(raw_html)

for text, pred in zip(test, test_pred):
    display_html(HTML(get_raw_html(text, pred)))

Н у п и з д е ц . С п а с и б о П у т и н у з а э т о ! С к о р о и н а ш и к о р ы т а т а к ж е з а п р е т я т . О ч е н ь д а ж е н е и с к л ю ч е н о

и б о d e b i t т а к ж е м о ж е т о з н а ч а т ь з а д о л ж е н н о с т ь Н е м о ж е т . И м е ю т с я в в и д у м а э с т р о и п р о ч а я х у й н я

е с л и ч е с т н о , т о х з , н о я б у д у б л а г о д а р е н , е с л и р а с с ш ь , к а к к а ч а т ь с к а ж е я н а х 1 0 0 п а р а ш а х . И б о я п р с т о к в е с т ы д е л а ю , а ш м о т в с ё т о т ж е . И я р а н о и л и п о з д н о п р о с т о у м и р а ю о т а т а к м о б о в

П р е д ы д у щ и й т р е д с б о к а л о м о х л а ж д е н н о г о а м а с е к а р а с с у ж д а е т о м а с с о в ы х б а т а л и я х , о т с у т с т в и и б а л а н с а и о с о б е н н о с т я х п р и м е н е н и я д е й с т в и й с п о д т и п о м р е а к ц и я . П р о д о л ж а е м р а с с к а з ы в а т ь с в о и и с т о р и и п р о т о , к а к а к о л и т ы и с к а л и к у л ь т и с т о в в п о д у л ь е и б ы л и х л а д н о к р о в н о у б и т ы а к о л и т а м и к о н к у р и р у ю щ е г о и н к в и з и т о р а , п р о э п и ч н о е п р е в о з м о г а н и е в р а г о в д е с а н т н и к а м и К а р а у л а С м е р т и , п р о п р и к л ю ч е н и я В о л ь н ы х Т о р г о в ц е в , п р о х а о с и т о в в е д у щ и х Ч е р н ы й К р е с т о в ы й П о х о д с о в м е с т н о с р а д и к а л ь н ы м и и н к в и з и т о р а м и и п р о г в а р д е й ц е в , п а ч к а м и д о х н у щ и х в O n l y W a r . С п о р и м о п р е и м у щ е с т в а х и н е д о с т а т к а х р а з н ы х с и с т е м , р е д а к ц и й и п е р е в о д о в , д е л и м с я с т а р ы м и O E F к а м и и с н е д о в е р и е м ж д е м в ы х о д а W r a t h G l o r y , а т а к ж е о б е щ а е м з а в т р а з а п и л и т ь р е п о р т ы . Г а й д Д Х - т р е д а , в е р с и я 1 . 1 : С с ы л к а н а к н и г и : Е щ е о д н а с с ы л к а н а к н и г и : К н и г и в п е р е в о д е э н т у з и а с т о в в а р ф о р д ж а : Т о т ж е п е р е в о д , н о в ф о р м а т е в и к и и : С с ы л к а н а а л ь т е р н а т и в н ы й р у с е к : т у т м о г л а б ы т ь в а ш а р е к л а м а с а й т а с х о у м р у л а м и Р а с с у ж д е н и я о с о ц и у м е 4 0 к : В в о д н а я в о в т о р у ю р е д а к ц и ю д л я м а н ч к и н о в :

V I R U S D O N T C L I C K ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !

...

З а б ы л - и к а к о й н а б о р д л я н о ч ё в к и ?

Д Н Н - д е н ь г и н е н у ж н ы . С р а з у с к а ж у - э т о н е л е в а я и д е а л о г и я , н е с о ц и а л и з м и н е к о м м у н и з м . Н а п и к а х л а й ф х а к , к а к н а е б а т ь с и с т е м у и б ы т ь п о б е д и т е л е м п о ж и з н и б е з д е н е г , х а й п а ж о р с т в а и п о п у л я р н о с т и у ш и р н а р м а с с 0 ) Б о л ь ш е н е н у ж н о с о р е в н о в а т ь с я з а л и д е р с т в о , х а й п , з а л у п а й п , у с т р а и в а т ь с р а ч и и х о л и в а р ы , в е д ь э т о и с т и н н о н а р о д н а я д е м о к р а т и я п р и к о т о р о й в с е ч л е н ы р а в н ы . Р а б о т а н е н у ж н а ( Р Н Н ) - т е п е р ь э т о н е п р о с т о г р о м к и е с л о в а . В с ё с в о б о д н о е в р е м я м о ж н о п о с в я т и т ь с а м о р а з в и т и ю л и ч н о с т и . 1 ) Н у л е в а я с е б е с т о и м о с т ь 2 ) В ы с о к о к а ч е с т в е н н ы й , п и т а т е л ь н ы й б е л о к , п р а к т и ч е с к и к р е м л ё в к а к т о н е м о ж е т б е з у г л е й - д о б а в ь т е в с у п и з о д у в а н ч и к о в и и л и к р а п и в ы 3 ) Д о с т у п н о е р е ш е н и е , е с т ь в л ю б о м г о р о д е 4 ) П р и и з б ы т к е м о ж н о м е н я т ь н а ш м о т , л у т , т я н и т п с р е д и п о с л е д о в а т е л е й д в и ж е н и я и з а е г о п р е д е л а м и Н а с у ж е м и л л и о н ы п о в с е м у м и р у - п р и с о е д и н я й т е с ь

Н е с м о т р я н а т ё м н ы е в р е м е н а , м ы , и с т и н н ы е ф а н а т ы Ч е л с и , н е о т в е р н ё м с я о т н а ш е г о в е л и к о г о к л у б а . В п е р е д и е щ ё б о р ь б а з а л и г о ч е м п и о н с к у ю ч е т в ё р к у и К у б о к А н г л и и . В е р и м в к о м а н д у и К о н т е и л и к а к о г о - н и б у д ь д р у г о г о т р е н е р а , е с л и в с ё ж е у в о л я т и п ы т а е м с я н е п р о п у с к а т ь м е ж д у н о г . З а л е т а й т е , ф у л х э м с к и е л ь в ы . П р е д ы д у щ и й т р е д , п о л н ы й б о л и и с р а ч е й , н а х о д и т с я т у т

С а п с н а ч , ч т о в ы в и д и т е н а э т о й в е б м . А т о м ы с т о в а р и щ е м и з б в и д и м а б с о л ю т н о р а з н ы е в е щ и

П а р т и я ц в е т н ы х л о ш а д о к н а х о д и т с я п е р е д в х о д о м в п е щ е р у . З л о в е щ а я т ь м а п о д з е м е л ь я м а н и т , о б е щ а я о г р о м н ы е б о г а т с т в а и л и у ж а с н у ю с м е р т ь . О с м е л и т е с ь л и в ы п р и н е с т и с в е т д р у ж б о м а г и и в э т о г и б л о е м е с т о ?